Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
import pandas as pd
sales = pd.read_csv("sales_train.csv")
fail_sale = sales[(sales['item_cnt_day'] > 0)]
product = fail_sale.groupby(by=['item_id']).sum().groupby(level=[0]).cumsum().sort_values(by='item_cnt_day', ascending=False).head(10)
product[['item_cnt_day']]

,item_cnt_day
item_id,
20949,187660.0
2808,17255.0
3732,16676.0
17717,15830.0
5822,14522.0
3734,11733.0
6675,10315.0
3731,10105.0
1855,10041.0


In [ ]:
date_2013 = fail_sale[(pd.to_datetime(fail_sale.date).dt.year) == 2013]
date_2014 = fail_sale[(pd.to_datetime(fail_sale.date).dt.year) == 2014]
date_2015 = fail_sale[(pd.to_datetime(fail_sale.date).dt.year) == 2015]
valid_date = date_2013[date_2013['item_id'].isin(date_2014['item_id']) & date_2013['item_id'].isin(date_2015['item_id'])]
top = fail_sale.groupby(by=['item_id'])['item_cnt_day'].sum().reset_index().sort_values(by='item_cnt_day', ascending=False).head(10)
top[top["item_id"].isin(valid_date["item_id"])]

,item_id,item_cnt_day
20599,20949,187660.0
2749,2808,17255.0
3654,3732,16676.0
17416,17717,15830.0
5717,5822,14522.0
3656,3734,11733.0
6543,6675,10315.0
1814,1855,10041.0
16491,16787,9255.0


# Задание 2

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

In [ ]:
item  = pd.read_csv("items.csv")
category = pd.read_csv("item_categories.csv")
use_items = sales.join(item, on='item_id', lsuffix='_new').join(category, on='category_id', lsuffix='_new')
use_items = use_items[(use_items['item_cnt_day'] > 0)]
min_category = use_items.groupby(['item_category_id','item_category_name'])['item_cnt_day'].sum().reset_index().sort_values('item_cnt_day').reset_index().head(1)
min_category[['item_category_id','item_category_name']]

,item_category_id,item_category_name
0,51,Books - Cognitive literature


In [ ]:
max_category = use_items.groupby(['item_category_id','item_category_name'])['item_cnt_day'].sum().reset_index().sort_values(by='item_cnt_day', ascending=False).reset_index().head(1)
max_category[['item_category_id','item_category_name']]

,item_category_id,item_category_name
0,40,Cinema - DVD


# Задание 3

1. Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами.
2. Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.


In [ ]:
use_categories = use_items[(use_items['category_id']==51) | (use_items['category_id']==40)]
top5_shops = use_categories.groupby(['shop_id'])['item_cnt_day'].sum().reset_index().sort_values(by='item_cnt_day', ascending=False).head(5)
category_sales = use_categories[use_categories['shop_id'].isin(top5_shops['shop_id'])]
category_sales.groupby(['shop_id','item_id'])['item_cnt_day'].mean().reset_index().rename(columns={'item_cnt_day':'average_sales'})

,shop_id,item_id,average_sales
0,25,18,1.000000
1,25,19,1.000000
2,25,30,1.480620
3,25,32,1.263158
4,25,34,1.000000
...,...,...,...
19529,57,22152,1.100000
19530,57,22159,1.000000
19531,57,22160,1.000000
19532,57,22162,1.396226


In [ ]:
#c максимальными продажами
use_items['date'] = pd.to_datetime(use_items['date'])
month_table = use_items.assign(month = use_items.date.dt.month).reset_index(drop=True) 
month_sales = month_table.groupby(['month', 'item_id'])['item_cnt_day'].sum().reset_index()
month_sales_for_top5_shops = month_sales[month_sales['item_id'].isin(category_sales['item_id'])]
max_sale_for_item = month_sales_for_top5_shops.groupby(['item_id']).max().reset_index()
max_sale_for_item

,item_id,month,item_cnt_day
0,0,1,1.0
1,2,12,1.0
2,3,8,1.0
3,4,9,1.0
4,5,12,1.0
...,...,...,...
4711,22157,9,2.0
4712,22159,12,6.0
4713,22160,12,10.0
4714,22162,12,260.0


In [ ]:
#c минимальными продажами
min_sale_for_item = month_sales_for_top5_shops.groupby(['item_id']).min().reset_index()
min_sale_for_item 

,item_id,month,item_cnt_day
0,0,1,1.0
1,2,8,1.0
2,3,5,1.0
3,4,9,1.0
4,5,12,1.0
...,...,...,...
4711,22157,2,1.0
4712,22159,2,1.0
4713,22160,1,1.0
4714,22162,1,20.0


# Задание 4

1. Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
mean_table_for_category = use_categories.groupby(['date_block_num', 'category_id'])['item_price'].mean().reset_index()
mean_table_for_category

,date_block_num,category_id,item_price
0,0,40,244.474151
1,1,40,252.148935
2,2,40,260.664137
3,3,40,254.843725
4,4,40,247.604093
5,5,40,245.834224
6,6,40,236.750323
7,7,40,234.433452
8,8,40,241.795889
9,9,40,247.517896


In [ ]:
median_table_for_category = use_categories.groupby(['date_block_num', 'category_id'])['item_price'].median().reset_index()
median_table_for_category

,date_block_num,category_id,item_price
0,0,40,199.00
1,1,40,203.00
2,2,40,231.00
3,3,40,199.00
4,4,40,199.00
5,5,40,199.00
6,6,40,149.00
7,7,40,149.00
8,8,40,198.00
9,9,40,199.00


In [ ]:
#2013
use_year_2013 = mean_table_for_category[(mean_table_for_category['date_block_num'] >= 0) & (mean_table_for_category['date_block_num'] <= 11)]
table_for_2013 = []
for i in range(1, len(use_year_2013)-2):
  if(use_year_2013['item_price'][i] < use_year_2013['item_price'][i-1] and use_year_2013['item_price'][i]<use_year_2013['item_price'][i+1]):
     row = pd.DataFrame({'category_id':[use_year_2013['category_id'][i]], 'item_price' : [use_year_2013['item_price'][i]], 'month':[use_year_2013['date_block_num'][i]]})
     table_for_2013.append(row)
table_for_2013

[   category_id  item_price  month
 0           40  234.433452      7]

In [ ]:
#2014
use_category_40 = use_items[(use_items['category_id']==40)]
mean_table_for_category_40 = use_category_40.groupby(['date_block_num', 'category_id'])['item_price'].mean().reset_index()
use_year_2014 = mean_table_for_category_40[(mean_table_for_category_40['date_block_num'] >= 12) & (mean_table_for_category_40['date_block_num'] <= 23)].reset_index(drop=True)
table_for_2014 = []
for i in range(1, len(use_year_2014)-2):
  if(use_year_2014['item_price'][i] < use_year_2014['item_price'][i-1] and use_year_2014['item_price'][i]<use_year_2014['item_price'][i+1]):
     row = pd.DataFrame({'category_id':[use_year_2014['category_id'][i]], 'item_price' : [use_year_2014['item_price'][i]], 'month':[use_year_2014['date_block_num'][i]]})
     table_for_2014.append(row)
table_for_2014

[   category_id  item_price  month
 0           40   254.44539     16,    category_id  item_price  month
 0           40  263.874311     21]

In [ ]:
#2015
use_year_2015 = mean_table_for_category[(mean_table_for_category['date_block_num'] >= 24) & (mean_table_for_category['date_block_num'] <= 33)].reset_index(drop=True)
table_for_2015 = []
for i in range(1, len(use_year_2015)-2):
  if(use_year_2015['item_price'][i] < use_year_2015['item_price'][i-1] and use_year_2015['item_price'][i]<use_year_2015['item_price'][i+1]):
     row = pd.DataFrame({'category_id':[use_year_2015['category_id'][i]], 'item_price' : [use_year_2015['item_price'][i]], 'month':[use_year_2015['date_block_num'][i]]})
     table_for_2015.append(row)
table_for_2015

[   category_id  item_price  month
 0           40  256.377778     25,    category_id  item_price  month
 0           40  273.180515     31]